In [1276]:
import numpy as np
import random
import math
from typing import Literal

In [1277]:
#Types
Tile = str
Coordinates = tuple[int, int]
Up = tuple[Literal[-1], Literal[0]]
Down = tuple[Literal[1], Literal[0]]
Left = tuple[Literal[0], Literal[-1]]
Right = tuple[Literal[0], Literal[1]]
Direction = Up | Down | Left | Right
Compatibility = tuple[Tile, Tile, Direction]
Weights = dict[Tile, int]

UP = (-1, 0)
DOWN = (1, 0)
LEFT = (0, -1)
RIGHT = (0, 1)
DIRS = [UP, DOWN, LEFT, RIGHT]

# What we will do in this notebook ?
- Parse an input data to create rulesets for tiles
- Apply WFC to create a simple tiled version of WFC

In [1278]:
class Cell:

    def __init__(self, pos, options: set[Tile]):
        self.position = pos # position is given as a (y, x) tuple
        self.options = options # list of possible options available
        self.collapsed = False # collapsed state of the cell
        self.value = None # chosen option for this cell

    def __repr__(self):
        return f'tile {self.position} \n' \
               f'options : {[i for i in self.options]} \n' \
               f'value : {self.value} \n' \
               f'collapsed : {self.collapsed} \n'

    def collapse(self, choice):
        self.value = choice
        self.collapsed = True
        self.options = set(self.value)

    def constrain(self, forbidden:Tile):
        self.options.remove(forbidden)



# Parsing
- Looking at every possible combinations in a NxN grid
- Returns a list of possible rules

Rules are represented by a three element tuple : (elem1, elem2, direction)
Each rule should come with its counterpart from the other side

In [1279]:
input_matrix = np.array([
    ['L','L','L','L'],
    ['L','L','L','L'],
    ['L','L','L','L'],
    ['L','C','C','L'],
    ['C','S','S','C'],
    ['S','S','S','S'],
    ['S','S','S','S'],
], dtype=Tile)

In [1280]:
def valid_dirs(cur_coords:Coordinates, matrix_size:tuple[int, int]) -> list[Direction]:
    """
    Returns the valid directions from `cur_co_ord` in a matrix
    of `matrix_size`. Ensures that we don't try to take step to the
    left when we are already on the left edge of the matrix.
    :param cur_coords: the current coords
    :param matrix_size: the size of the matrix
    :return: a list of possible directions
    """
    y, x = cur_coords
    dirs : list[Coordinates] = []
    height, width = matrix_size

    if y > 0: dirs.append(UP)
    if y < height - 1 : dirs.append(DOWN)
    if x > 0: dirs.append(LEFT)
    if x < width - 1 : dirs.append(RIGHT)

    return dirs


In [1281]:
def parseExampleMatrix(matrix: np.ndarray) -> tuple[Weights, set[Compatibility], list[Tile]]:
    """
    Parses an example matrix. It extracts :
    1. Tile compatibilities
    2. The weight of each tile type

    :param matrix: a 2D matrix of tiles
    :return: A tuple :
        * A set of compatible tile combinations
        * A dict of weights for each tile type
    """
    compatibilities : set[Compatibility] = set()
    weights: Weights = {}
    options = list(set(input_matrix.flatten()))

    for y, row in enumerate(matrix):
        for x, cur_tile in enumerate(row):
            if cur_tile not in weights:
                weights[cur_tile] = 0
            weights[cur_tile] += 1

            for d in valid_dirs((y, x), matrix.shape):
                other = matrix[y+d[0], x+d[1]]
                compatibilities.add((cur_tile, other, d)) # Tile1, Tile2, Direction relative

    return weights, compatibilities, options

In [1282]:
parseExampleMatrix(input_matrix)

({'L': 14, 'C': 4, 'S': 10},
 {('C', 'C', (0, -1)),
  ('C', 'C', (0, 1)),
  ('C', 'L', (-1, 0)),
  ('C', 'L', (0, -1)),
  ('C', 'L', (0, 1)),
  ('C', 'S', (0, -1)),
  ('C', 'S', (0, 1)),
  ('C', 'S', (1, 0)),
  ('L', 'C', (0, -1)),
  ('L', 'C', (0, 1)),
  ('L', 'C', (1, 0)),
  ('L', 'L', (-1, 0)),
  ('L', 'L', (0, -1)),
  ('L', 'L', (0, 1)),
  ('L', 'L', (1, 0)),
  ('S', 'C', (-1, 0)),
  ('S', 'C', (0, -1)),
  ('S', 'C', (0, 1)),
  ('S', 'S', (-1, 0)),
  ('S', 'S', (0, -1)),
  ('S', 'S', (0, 1)),
  ('S', 'S', (1, 0))},
 ['C', 'L', 'S'])

# WaveFunction class
This class is responsible for handling the output grid

In [1283]:
class WaveFunction:

    def __init__(self, size:tuple[int, int], options, weights, comp:set[Compatibility]):
        self.grid = np.empty(shape = size, dtype=Cell)
        self.size = size
        self.weights: Weights = weights
        self.compatibilities = comp
        for _y in range(size[0]):
            for _x in range(size[1]):
                self.grid[_y, _x] = Cell((_y, _x), options)

    def __repr__(self):
        display = f""
        for _y in range(self.size[0]):
            for _x in range(self.size[1]):
                display += f" {str(self.grid[_y, _x].value):>4} "
            display += "\n"

        return display

    def get(self, coords:Coordinates):
        return self.grid[coords].options

    def check(self, Tile1, Tile2, dir):
        return (Tile1, Tile2, dir) in self.compatibilities

    def getEntropyAtCoords(self, coords:Coordinates):
        weights_sum = sum([self.weights[w] for w in self.get(coords)])
        weights_log_sum = sum([self.weights[w] +
                               math.log(self.weights[w])
                               for w in self.get(coords)])
        return math.log(weights_sum) - (weights_log_sum / weights_sum)

    def getMinEntropyCoords(self):
        min_entropy_coords: Coordinates = (0,0)
        min_entropy = None
        for y, row in enumerate(self.grid):
            for x, cell in enumerate(row):
                if self.grid[y, x].collapsed:
                    continue
                entropy = self.getEntropyAtCoords((y, x))
                # Add some noise
                entropy += (random.random() / 1000)
                if min_entropy is None or entropy < min_entropy :
                    min_entropy = entropy
                    min_entropy_coords = (y, x)

        return min_entropy_coords

    def collapseCellAtCoords(self, coords:Coordinates):
        cell:Cell = self.grid[coords]
        weighted_seq = [weight for (item, weight) in self.weights.items() if item in cell.options]
        choice = random.choices(list(cell.options), weighted_seq)
        cell.collapse(choice[0])


    def propagate(self, coords:Coordinates):
        stack = [coords]
        while len(stack) > 0:
            cur_coords = stack.pop() # get the current coords to analyze
            cur_possible_tiles = self.get(cur_coords) # get the available tiles
            #print(f"inspecting cell at coords {cur_coords}")
            # iterate around current coords
            for d in valid_dirs(cur_coords, self.size):

                other_coords = (cur_coords[0] + d[0], cur_coords[1] + d[1])
                #print(f"looking in coords {other_coords}")
                # loop through every tile in other location
                for other_tile in set(self.get(other_coords)):
                    #print(f"checking for tile {other_tile}")
                    # check if the other tile is compatible with any of the available tiles
                    # TODO: This doesnt work !!!!!!
                    other_tile_possible = any([
                        self.check(cur_tile, other_tile, d)
                        for cur_tile in cur_possible_tiles
                    ])
                    # If the tile is not compatible with any of the tiles in
                    # the current location then it is impossible
                    # for it to ever get chosen. We therefore remove it from
                    # the other location.
                    if not other_tile_possible:
                        #print(f"removing {other_tile}")
                        self.grid[other_coords].constrain(other_tile)
                        stack.append(other_coords)

    def getStateOfGrid(self):
        pass


# Test Zone

In [1284]:
class Model:

    def __init__(self, output_size, input_matrix):
        self.weights, self.compatibilities, self.options = parseExampleMatrix(input_matrix)
        self.wfc = WaveFunction(output_size, self.options, self.weights, self.compatibilities)

    def iterate(self):
        coords = self.wfc.getMinEntropyCoords()
        self.wfc.collapseCellAtCoords(coords)
        #self.wfc.propagate(coords)

        print(self.wfc)
        return coords

    def run(self):
        pass

In [1285]:
m = Model((3,3), input_matrix)


In [1286]:
m.wfc.grid[0, 0]

tile (0, 0) 
options : ['C', 'L', 'S'] 
value : None 
collapsed : False 

In [1287]:
c = m.iterate()

 None  None  None 
 None     S  None 
 None  None  None 



In [1288]:
for d in valid_dirs(c, (3,3)):
    other_coords = (c[0] + d[0], c[1] + d[1])
    #print(f"looking in coords {other_coords}")
    # loop through every tile in other location
    for other_tile in set(m.wfc.get(other_coords)):
        other_tile_possible = any([
                        m.wfc.check(cur_tile, other_tile, d)
                        for cur_tile in m.wfc.get(c)
                    ])
        print(f"direction {d}, {other_tile} : {other_tile_possible}")
        #if not other_tile_possible:
        #    m.wfc.grid[other_coords].constrain(other_tile)

direction (-1, 0), C : True
direction (-1, 0), L : False
direction (-1, 0), S : True
direction (1, 0), C : False
direction (1, 0), L : False
direction (1, 0), S : True
direction (0, -1), C : True
direction (0, -1), L : False
direction (0, -1), S : True
direction (0, 1), C : True
direction (0, 1), L : False
direction (0, 1), S : True
